# Capstone: Airbnb Price Listing Prediction

_Authors: Evonne Tham_


<div class="alert alert-block alert-danger">


<b>Checkin 3</b>

The actual deliverable for this section is fairly small. You are expected to submit, via this Google form, a progress report detailing where you are with your Capstone. The format and content is up to you but, at a minimum, we expect the following:


- Do you have data fully in hand and if not, what blockers are you facing?
- Have you done a full EDA on all of your data?
- Have you begun the modeling process? How accurate are your predictions so far?
- What blockers are you facing, including processing power, data acquisition, modeling difficulties, data cleaning, etc.? How can we help you overcome those challenges?
- Have you changed topics since your lightning talk? Since you submitted your Problem Statement and EDA? If so, do you have the necessary data in hand (and the requisite EDA completed) to continue moving forward?
- What is your timeline for the next week and a half? What do you have to get done versus what would you like to get done?
- What topics do you want to discuss during your 1:1?
- We will use your progress report as a leaping off point for a deep 1:1 with a member of the instructional staff. The best use of your time is to really flesh out this document so that your remaining time here can be most productive.

</div>

<div class="alert alert-block alert-warning">


<b>Checkin 2: Think about the following as you perform your initial EDA.</b>

- Examine the distributions of your data, numerically and/or visually.
- Identify outliers.
- ~~Identify missing data and look for patterns of missing data.~~
- Describe how your EDA will inform your modeling decisions and process.

</div>

## <span style='background:yellow'> 1. Import Necessary Libraries & Load Data </span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

# RdYlGn

#sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
country = 'japan/'
path = '../datasets/'

calendar = pd.read_csv(path + country + 'calendar.csv')
listing = pd.read_csv(path + country + 'listings.csv')
neighbourhood = pd.read_csv(path + country + 'neighbourhoods.csv')
reviews = pd.read_csv(path + country + 'reviews.csv')

In [ ]:
# # show all rows and columns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## <span style='background:yellow'> 2. Initial Exploratory Analysis </span>
Evaluate the current state of the data

##### Defining Functions

In [ ]:
def eda(dataframe):
    
    """Runs basic EDA on dataframe."""
    
    print("DF shape: ", dataframe.shape)
    print("")
    print("DF types:\n", dataframe.dtypes)
    print("***********************************************************************************")
    print("")
    print("DF describe: \n", dataframe.describe())
    print("***********************************************************************************")
    print("")
    print("DF describe: \n", dataframe.describe(exclude=np.number).T)
    print("***********************************************************************************")
    print("")
    print("Missing values: \n", 
          (dataframe.isnull().sum())[dataframe.isnull().sum()>0].sort_values(ascending=False))
    print("***********************************************************************************")
    print("")
    print("duplicate values: ", dataframe.duplicated().sum())
    print("***********************************************************************************")
    print("")
    #check if column is unique
    for i in dataframe.columns:
        print('{} is unique: {}'.format(i, dataframe[i].is_unique))
    print("***********************************************************************************")
    print("***********************************************************************************")
    print("")

### <span style='background:yellow'> 2.1. Calendar Dataset</span>

In [ ]:
eda(calendar)

In [ ]:
calendar.head()

In [ ]:
print(calendar['date'].min())
print(calendar['date'].max())

<div class="alert alert-block alert-success">

<b> Calendar Dataset Observations: </b>
- calendar ranged from 2020-05-30 to 21-05-30
- most likely t and f represents true and false
- columns that might be useful 
    - listing_id (to combine with other datasets)
    - date
    - available
    - price

<b> To-Dos: </b>
- convert date object to datetime
- remove $, from price column
- get dummy for available column
- replace t and f value to 1s and 0s
- fill Null values fro minimum and maximum nights
</div>

### <span style='background:yellow'> 2.2. Listing Dataset </span>

In [ ]:
eda(listing)

In [ ]:
listing.head().T

<div class="alert alert-block alert-success">

<b> Observations: </b>
- Dataset consist of 15009 listings 
- has over 100 features

<b> To-Dos: </b>
- To dropping columns (entirely made up of NaNs): 
    - neighbourhood_group_cleansed,
    - medium_url, 
    - thumbnail_url, 
    - xl_picture_url, 
    - jurisdiction_names
    - experiences_offered
- consider dropping columns(consist of only about 100 data):
    - square_feet 
    - monthly_price
    - weekly_price
- consider dropping columns that are text focused
- drop rows with mostly null value
- replace t and f value to 1s and 0s
- one hot coding for categorical variables, for example
    - cancellation policy ('strict_14_with_grace_period', 'moderate', 'flexible', 'strict',
       'super_strict_60', 'super_strict_30')
    - property_type
    - room_type
- clean columns with $ 
- review scores - imputate with median
</div>

### <span style='background:yellow'> 2.3. Neighbourhood Dataset </span>

In [ ]:
eda(neighbourhood)

In [ ]:
neighbourhood.head()

<div class="alert alert-block alert-success">

<b> Neighbourhood Dataset Observations: </b>
- fairly simple dataset 
- total 62 neighbourhood
- might not need this dataset for analysis

<b> To-Dos: </b>
- to drop neighbourhood_group column (fully NaNs)
</div>

### <span style='background:yellow'> 2.4. Reviews Dataset </span>

In [ ]:
eda(reviews)

In [ ]:
reviews.head()

In [ ]:
print(reviews['date'].min())
print(reviews['date'].max())

<div class="alert alert-block alert-success">

<b> Reviews Dataset Observations: </b>
- there are 398181 reviews
- dated from 2011-09-21 to 2020-05-30
- 280 missing comments

<b> To dos: </b>

- convert date object to datetime
- clean comments

</div>

## <span style='background:yellow'> 3. Data Cleaning </span>

##### Defining Functions

In [ ]:
def mass_cleaning(df):
    
    """Clean date and replace ts and fs"""
    
    # Parse date-time string to datetime object
    if 'date' in df.columns: 
        df['date'] = pd.to_datetime(df['date'])

    
    #Columns with Ts and Fs
    df.replace({'f': 0, 't': 1}, inplace=True)
    
    # Clean Currency
    df = df.applymap(lambda x: int(x.replace('$','').replace(',','').replace('.00','')) 
                     if (type(x) == str) and (x[0]=='$') else x )
    
    return df

In [ ]:
def impute_median(df, column):
    
    """ replace NaN values with median"""
    
    df[column].fillna((df[column].median()), inplace=True)

### <span style='background:yellow'> 3.1. Calendar </span>

In [ ]:
calendar = mass_cleaning(calendar)
calendar.head()

In [ ]:
# Imputate missing values in minimum maximum nights with 1s, 0s 
calendar['minimum_nights'].fillna(0, inplace=True)
calendar['maximum_nights'].fillna(0, inplace=True)

# Replace continuous to discrete
calendar['minimum_nights'] = calendar['minimum_nights'].astype(int)
calendar['maximum_nights'] = calendar['maximum_nights'].astype(int)

#Check for nulls
calendar.isnull().sum()

<div class="alert alert-block alert-warning">
<b>Note to self:</b>
^ Might consider dropping `minimum_nights` and  `maximum_nights`
</div>

In [ ]:
plt.figure(figsize=(15,5))
calendar['price'].plot.line(alpha=0.5, color = '#FF5A5F')
calendar['adjusted_price'].plot.line(alpha=0.5, color = '#00A699')
plt.ylabel('$ ', fontsize = 25, rotation=0)
plt.legend(['Price', 'Adjusted Price']);

<div class="alert alert-block alert-warning">

<b>Note to self:</b>
Doesnt seem to have much difference in majority of the lising, can consider dropping `adjusted_price`?


</div>

##### Dropping Columns

<div class="alert alert-block alert-warning">

<b>Note to self:</b>
To remove lines in line18, if dropping cols

</div>

In [ ]:
# cols_to_drop = ['adjusted_price', 'minimum_nights', 'maximum_nights']

# calendar = calendar.drop(cols_to_drop, axis=1)
# calendar.shape

##### Save DataFrame

In [ ]:
calendar.to_csv("../datasets/calendar_cleaned.csv")

### <span style='background:yellow'> 3.2. Listing </span>

In [ ]:
listing = mass_cleaning(listing)

##### Isolate and inspect categorical and numerical variable

<div class="alert alert-block alert-success">
As the the number of features is fairly large, I will be splitting them up by categorical and numerical.
</div>

In [ ]:
categorical_df = listing.select_dtypes(include=['object', 'bool'])
categorical_cols = listing.select_dtypes(include=['object', 'bool']).columns.values

In [ ]:
categorical_df.head(2).T

In [ ]:
numerical_df = listing.select_dtypes(include=['int', 'float'])
numerical_cols = listing.select_dtypes(include=['int', 'float']).columns.values

In [ ]:
numerical_df.head(2).T

### 3.2.1. Dropping  and Convert Columns

<div class="alert alert-block alert-success">
    
<b>Explanation:</b>

From the data shown above (categorical/numerical), I will be dropping text columns for now and columns that might not be useful for predicting price. 
</div>

In [ ]:
cols_to_drop = ['listing_url', 'last_scraped', 'name', 'summary','space', 
                'description', 'neighborhood_overview', 'notes', 'transit', 'access', 
                'interaction', 'house_rules', 'picture_url', 'host_url','host_name', 
                'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url','host_neighbourhood', 
                'host_verifications', 'calendar_last_scraped', 'license', 'scrape_id','thumbnail_url',
                'medium_url', 'xl_picture_url'
               ]

listing = listing.drop(cols_to_drop, axis=1)
listing.shape

In [ ]:
# Convert rest of the datetime string column to datetime object
listing['host_since'] = pd.to_datetime(listing['host_since'])
listing['first_review'] = pd.to_datetime(listing['first_review'])
listing['last_review'] = pd.to_datetime(listing['last_review'])

In [ ]:
# Check for columns that has more than 10000 Null values
listing.isnull().sum()[listing.isnull().sum()>10000].sort_values(ascending=False)

In [ ]:
# Drop columns consist a majority of NaNs 
# reviews_per_month and number_of_reviews_ltm columns will be dropped as well as there is number_of_reviews 
# experiences_offered are all 'none' values as well
listing.drop(['jurisdiction_names','neighbourhood_group_cleansed','square_feet',
              'monthly_price','weekly_price', 'reviews_per_month','number_of_reviews_ltm', 
              'experiences_offered'], 
             axis=1, inplace=True)

In [ ]:
listing.shape

### 3.2.2  Imputation of Missing Values

In [ ]:
listing.isnull().sum()[listing.isnull().sum()>1000].sort_values(ascending=False)

<div class="alert alert-block alert-success">
    
<b>Assumption:</b>

Missing data from (cols that start with name): 

- review_scores
- host_response
- last_review 
- first_review
- host_acceptance_rate

Might be an indication of properties that are new listing that has not been contacted yet. Hence they do not have any response time and rate, and/or ratings . Hence, null values for these features will be filled with `unrated`, `unknown` accordingly. 

I will also be splitting them into categories.
- reviews_scores does not have all the rating from 0-10, hence I will categorise them according to 
    - low 
    - acceptable 
    - good
    - very good 
    - excellent

</div>

##### a. Review_Scores_

In [ ]:
print('Unique Review Scores Value: ', listing.review_scores_value.unique())
print('**************************************************************')
print('Unique Review Scores Location: ', listing.review_scores_location.unique())
print('**************************************************************')
print('Unique Review Scores Comm: ', listing.review_scores_communication.unique())
print('**************************************************************')
print('Unique Review Scores Checkin: ', listing.review_scores_checkin.unique())
print('**************************************************************')
print('Unique Review Scores Cleaniness: ', listing.review_scores_cleanliness.unique())
print('**************************************************************')
print('Unique Review Scores Acc: ', listing.review_scores_accuracy.unique())
print('**************************************************************')
print('Unique Review Scores Rating: ', listing.review_scores_rating.unique())

In [ ]:
listing.review_scores_value.value_counts()

##### Base on the the distribution of the scores, I will create bins accordingly.

In [ ]:
for col in listing.columns:
    if (col in listing.columns[listing.columns.str.startswith('review_scores_')]) and not (col in listing.columns[listing.columns.str.endswith('_rating')]):
        
        # Create Categories
        bins = [0,3,7,8,9,10]
        labels = ['0-3','4-7','8','9','10']
        
        listing[col] = pd.cut(listing[col],
                                      bins=bins, 
                                      labels=labels, 
                                      include_lowest= True)
        
        # Create unrated Category for 'NaN'
        listing[col] = (listing[col].cat.add_categories('unrated').fillna('unrated'))
        
listing.review_scores_value.value_counts()

In [ ]:
listing.review_scores_rating.value_counts()

In [ ]:
# Create Categories
bins = [0, 50, 90, 99, 100]
labels = ['0-49%', '50-89%', '90-99%', '100%']

listing['review_scores_rating'] = pd.cut(listing['review_scores_rating'],
                                      bins=bins, 
                                      labels=labels, 
                                      include_lowest= True)

# Create Unknown Category for 'NaN'
listing['review_scores_rating'] = (listing['review_scores_rating'].cat
                                                            .add_categories('unrated')
                                                            .fillna('unrated'))

listing.review_scores_rating.value_counts()

In [ ]:
# # Imputate null values with 'Unknown' for column that
# # starts with review_scores_ and ends with _review
# for col in listing.columns:
#     if (col in listing.columns[listing.columns.str.startswith('review_scores_')]): #& (col in listing.columns[listing.columns.str.endswith('_review')]):
#         listing[col].fillna('unrated', inplace = True)

In [ ]:
listing.isnull().sum()[listing.isnull().sum()>0].sort_values(ascending=False)

##### b. _Reviews

<div class="alert alert-block alert-warning">
    
- Not sure what to do with this.
- fillna with ??? 
- drop last and first review??? 

- BRB

</div>

In [ ]:
listing.first_review.dtype

In [ ]:
# last and first reviews
listing['time_since_first_review'] = listing.last_review - listing.first_review
# listing.time_since_first_review.hist(figsize=(11,9), bins=30);

In [ ]:
listing['time_since_first_review'].value_counts()

In [ ]:
listing.isnull().sum()[listing.isnull().sum()>0].sort_values(ascending=False)

##### b. Host 
_i. Response time_

In [ ]:
listing.host_response_time.value_counts()

In [ ]:
listing.host_response_time.fillna("unknown", inplace=True)
listing.host_response_time.value_counts(normalize=True)

_ii. Response rate_

In [ ]:
# Host response rate
listing.host_response_rate.unique()

In [ ]:
# Replace Unneccessary Character and convert string to float
listing.host_response_rate = listing.host_response_rate.str.replace('%','')
listing.host_response_rate = listing.host_response_rate.astype('float64')

# Create Categories
bins = [0, 50, 90, 99, 100]
labels = ['0-49%', '50-89%', '90-99%', '100%']

listing['host_response_rate'] = pd.cut(listing['host_response_rate'],
                                      bins=bins, 
                                      labels=labels, 
                                      include_lowest= True)

listing.host_response_rate.value_counts()

In [ ]:
# Create Unknown Category for 'NaN'
listing['host_response_rate'] = (listing['host_response_rate'].cat
                                                            .add_categories('unknown')
                                                            .fillna('unknown'))

listing.host_response_rate.value_counts()

_iii. Acceptance rate_

In [ ]:
listing.host_acceptance_rate.unique()

In [ ]:
# Replace Unneccessary Character and convert string to float
listing.host_acceptance_rate = listing.host_acceptance_rate.str.replace('%','')
listing.host_acceptance_rate = listing.host_acceptance_rate.astype('float64')

# Create Categories
bins = [0, 50, 90, 99, 100]
labels = ['0-49%', '50-89%', '90-99%', '100%']

listing['host_acceptance_rate'] = pd.cut(listing['host_acceptance_rate'],
                                      bins=bins, 
                                      labels=labels, 
                                      include_lowest= True)

listing.host_acceptance_rate.value_counts()

In [ ]:
# Create Unknown Category for 'NaN'
listing['host_acceptance_rate'] = (listing['host_acceptance_rate'].cat
                                                            .add_categories('unknown')
                                                            .fillna('unknown'))

listing.host_acceptance_rate.value_counts()

In [ ]:
listing.isnull().sum()[listing.isnull().sum()>0].sort_values(ascending=False)

##### c. Address Related

<div class="alert alert-block alert-success">
    
<b>Observation:</b>

Looking at the categorical_df dataset. There are a couple of columns that are address related 
- street
- neighbourhood
- neighbourhood_cleansed --> to keep (Since the column is kept, Neighbourhood Dataset will not be required
- city
- state
- zipcode
- market
- smart_location
- country_code --> to keep
- country

From numerical_df, to safe in a new dataframe for later use
- longitute 
- latitude

</div>

In [ ]:
# To Check and Compare Address Related Columns

# print(listing.street.value_counts())
# print(listing.street.nunique())
# print(listing.neighbourhood_cleansed.value_counts())
print(listing.neighbourhood_cleansed.nunique())
# print(listing.city.value_counts())
# print(listing.city.nunique())
# print(listing.state.value_counts())
# print(listing.state.nunique())
# print(listing.market.value_counts())
# print(listing.market.nunique())
# print(listing.smart_location.value_counts())
# print(listing.smart_location.nunique())
# print(listing.country_code.value_counts())
# print(listing.country_code.nunique())

# listing[listing.state == 'Shibuya']['neighbourhood']

In [ ]:
long_lat_df = listing[['longitude','latitude']]
long_lat_df.to_csv('../datasets/long_lat_df.csv')

In [ ]:
listing.drop(['street','neighbourhood', 'city','state','zipcode',
              'market','smart_location','country','longitude','latitude'], 
             axis=1, inplace=True)

# Check
listing.isnull().sum()[listing.isnull().sum()>0].sort_values(ascending=False)

##### d. Fees

<div class="alert alert-block alert-success">
    
<b>Assumption:</b>

Hosts can opt for no security_deposit or cleaning_fee. hence, will imputate null values as `0`

</div>

In [ ]:
listing.security_deposit.fillna(0, inplace=True)
listing.cleaning_fee.fillna(0, inplace=True)

listing.isnull().sum()[listing.isnull().sum()>0].sort_values(ascending=False)

##### e. Beds,  Bedrooms , Bathrooms and Host_

<div class="alert alert-block alert-success">
    
<b>Assumption:</b>

It is very usual for a listing to have no beds, and/or bathrooms. On top of that, a fully signed up Host on Airbnb will have a date of signing up. Hence I will be checking if they are of the same before i drop them. As for bedrooms, there is a possility that it is a studio apartment however, i cant entire assume. To avoid strange fractions.

</div>

In [ ]:
# Beds and Bathrooms
listing.dropna(subset=['beds','bathrooms'], inplace=True)
impute_median(listing, 'bedrooms')

In [ ]:
# Host_
# Checking if the 7 rows are of the same 
listing[listing['host_identity_verified'].isnull()| 
        listing['host_has_profile_pic'].isnull() | 
        listing['host_total_listings_count'].isnull() |
        listing['host_listings_count'].isnull() |
        listing['host_is_superhost'].isnull() | 
        listing['host_since'].isnull()]

In [ ]:
listing.dropna(inplace=True)
listing.shape

#### 3.2.3  Individual Columns

##### a. Listing Count

In [ ]:
print('Total number of listings: ', listing.shape[1])
print('Differences between columns: ', sum((listing.host_listings_count == listing.host_total_listings_count) == False))

<div class="alert alert-block alert-success">

<b>Observations:</b>

Host listing count and host total listings counts seem to have the same data. There are also other columns that are similar to listing counts (which are split up into multiple columns:

- calculated_host_listings_count, 
- calculated_host_listings_count_entire_homes, 
- calculated_host_listings_count_private_rooms, 
- calculated_host_listings_count_shared_rooms

Hence these columns will be dropped as well.
</div>

In [ ]:
listing.drop(['host_total_listings_count','calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
              'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms'], 
             axis=1, inplace=True)

##### b. Boolean Columns

In [ ]:
# Columns with only booleans for Verfication Purposes:
listing[['host_is_superhost',
         'host_has_profile_pic', 
         'host_identity_verified', 
         'require_guest_profile_picture',
         'require_guest_phone_verification',
         'requires_license', 
         'instant_bookable', 
         'is_business_travel_ready', 
         'is_location_exact',
         'has_availability'
        ]].hist(figsize=(20,10), color='#FF5A5F');

<div class="alert alert-block alert-success">

<b>Observation:</b>

As shown above some of the features showed uniform result. Hence will be dropping them as they will not be affect the price. 
- host_has_profile_pic
- is_business_travel_ready
- require_guest_phone_verification
- require_guest_profile_picture
- requires_license

</div>

In [ ]:
listing.drop(['host_has_profile_pic',
              'is_business_travel_ready',
              'require_guest_phone_verification',
              'require_guest_profile_picture',
              'requires_license', 
              'has_availability'], axis=1, inplace=True)

##### c. Minimum and Maximum Nights

<div class="alert alert-block alert-success">

<b>Observation:</b>

There are multiple columns that are related to minimum and maximum night stays. To avoid overlapping of data, I will using the basic two which have not been altered.

</div>

In [ ]:
listing.drop(['minimum_minimum_nights',
             'maximum_minimum_nights',
             'minimum_maximum_nights',
             'maximum_maximum_nights',
             'minimum_nights_avg_ntm',
             'maximum_nights_avg_ntm'], axis=1, inplace=True)

##### d. Categorical Columns

##### i. property_type

<div class="alert alert-block alert-success">

<b>Observation:</b>

There are a total of 29 unique property types with a majority being the Apartment. However other property types (e.g. house, condominium, serviced apartment) can be considered to be in the same category as well. 

</div>

In [ ]:
print(listing.property_type.nunique())
listing.property_type.value_counts()

In [ ]:
# # Regroup and rename property type
# listing.property_type.replace({
#     'Apartment': 'house',
#     'House': 'house',
#     'Hostel': 'hostel',
#     'Condominium': 'apartment', 
#     'Aparthotel': 'hotel', 
#     'Boutique hotel': 'hotel', 
#     'Villa': 'house', 
#     'Serviced apartment': 'apartment',
#     'Ryokan (Japan)': 'apartment',
#     'Hut': 'house',
#     'Loft': 'apartment',
#     'Guesthouse': 'house',
#     'Bed and breakfast': '',
#     'Townhouse': 'house',
#     'Guest suite': '',
#     'Tiny house': 'house',
#     'Other': '',
#     'Cabin': '',
#     'Dome house': 'house',
#     'Camper/RV': '',
#     'Bungalow': 'house',
#     'Tent': '',
#     'Nature lodge': 'house',
#     'Resort','house',
#     'Earth house','',
#     'Cottage','house',
#     'Castle','castle',
#     'Dorm','hostel'
#     }, inplace=True)

##### ii. room_type

In [ ]:
listing.room_type.value_counts()

In [ ]:
# Rename room type
listing.room_type.replace({
    'Entire home/apt': 'entire_home_apt',
    'Private room': 'private_room',
    'Hotel room': 'hotel_room',
    'Shared room': 'shared_room',  
    }, inplace=True)

##### iii. bed_type

<div class="alert alert-block alert-success">

<b>Observation:</b>

This column seem kinda redundent since majority are the same bed. Hence will be dropping this column. 

</div>

In [ ]:
listing.bed_type.value_counts()

In [ ]:
listing.drop(['bed_type'], axis=1, inplace=True)

##### iv. amenities

<div class="alert alert-block alert-success">

Amenities is a list of additional features in the property where i believe people look at before (e.g. wifi, air conditioning, swimming pool, etc), committing the booking. However not all amenities are of the same importance.

There are over 150 unique type of amenities

</div>

In [ ]:
listing.amenities.head()

In [ ]:
listing['amenities'] = listing['amenities'].str.replace('{', '').str.replace('}','').str.replace('"', '')
listing['amenities'] = listing['amenities'].str.lower()

##### v. Calendar_updated

<div class="alert alert-block alert-warning">

<b>Observation:</b>

- to get back to this
- or else to be dropped

</div>

In [ ]:
print("Total categories:", listing.calendar_updated.nunique())
print('')
print("Top 5 categories:")
listing.calendar_updated.value_counts()[:5]

##### vi. Availability

<div class="alert alert-block alert-warning">

<b>Observation:</b>

- to get back to this
- or else to be dropped

</div>

In [ ]:
print("Avail 30days categories:", listing.availability_30.nunique())
print("Avail 60days categories:", listing.availability_60.nunique())
print("Avail 90days categories:", listing.availability_90.nunique())
print("Avail 365days categories:", listing.availability_365.nunique())
print('')
print("Top 5 categories of 30days:")
listing.availability_30.value_counts()[:5]

In [ ]:
print(listing.shape)
listing.dtypes

##### vi. Cancellation Policy

In [ ]:
listing.cancellation_policy.value_counts()

In [ ]:
listing.cancellation_policy.replace({
    'strict_14_with_grace_period': 'strict',
    'super_strict_30': 'super_strict',
    'super_strict_60': 'super_strict',
    'strict_14_with_grace_period': 'strict',
    'luxury_moderate': 'moderate'
    }, inplace=True)
listing.cancellation_policy.value_counts()

In [ ]:
print(listing.shape)
listing.head().T

In [ ]:
listing.dtypes

##### Save DataFrame

In [ ]:
# listing.to_csv("../datasets/listing_cleaned.csv")

##### Merge Dataframe

In [ ]:
# listing_cal = pd.merge(listing, calendar, how='left', on=['id', date','price'])

###  <span style='background:yellow'> 3.3. Neighbourhood </span>

In [ ]:
neighbourhood = neighbourhood.drop(['neighbourhood_group'], axis=1)
neighbourhood

<div class="alert alert-block alert-success">

<b>Observation:</b>
Since listing dataframe has the column on neighbourhood_cleansed and country code. This dataframe might not be required. 

</div>

##### Save DataFrame

In [ ]:
neighbourhood.to_csv("../datasets/neighbourhood_cleaned.csv")

### <span style='background:yellow'> 3.4 Reviews </span>

In [ ]:
reviews.head()

In [ ]:
# Parse date-time string to datetime object
reviews.date = pd.to_datetime(reviews['date'])

In [ ]:
# Drop Unneccessary Columns
reviews.drop(['id','reviewer_id','reviewer_name'], axis=1, inplace=True)

<div class="alert alert-block alert-success">

<b>Observation:</b>

In the comments column, there are languages other than english. RNN(?) 

</div>

##### Merge Dataframe

In [ ]:
# final = pd.merge(listing_cal, reviews, how='left', on=['listing_id'])

##### Save DataFrame

In [ ]:
# reviews.to_csv("../datasets/reviews_cleaned.csv")

### <span style='background:yellow'> 4. Full Exploratory Analysis </span>

### Listing Dataset

##### a. Price

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
listing.price.describe()

In [ ]:
plt.figure(figsize =(12,6))
sns.distplot(listing['price'], kde= False, bins=300, color ='#FF5A5F')
plt.ticklabel_format(style='plain', axis='y')
plt.ticklabel_format(style='plain', axis='x')
plt.tight_layout()
plt.ylabel('Frequency', fontsize=15)
plt.xlabel('Price(¥)', fontsize=15)
plt.title('Distribution of Listing Price', fontsize = 20);

In [ ]:
plt.figure(figsize =(12,6))
listing['price'].hist(bins=[0,5000,10000,15000,20000,25000,30000,35000,40000,45000,50000,55000,
                           60000,65000,70000,75000,80000,85000,90000,95000,100000], color ='#FF5A5F')
plt.tight_layout()
plt.title('Distribution of Listing Prices Below 100,000¥', fontsize=20)
plt.xlabel('Price(¥)', fontsize=15)
plt.ylabel('Frequency', fontsize=15);

In [ ]:
listing[listing['price'] == 0]
listing[listing['price'] > 100000].sort_values(by='price', ascending=False)

<div class="alert alert-block alert-success">

<b>Observation:</b>

There seem to have listing that are of zero value. On top of that, there are listing that are listed at outrages price

</div>

<div class="alert alert-block alert-warning">

<b>To Do:</b>

- to drop prices that are priced at zero 
- to drop outrages price listings

</div>

##### a. Review_Scores

In [ ]:
review_col = list(listing.columns[listing.columns.str.startswith("review_scores") == True])

fig = plt.figure(figsize=(12,8))
for i, var_name in enumerate(review_col):
    ax = fig.add_subplot(3,3,i+1)
    listing[var_name].hist(bins=10, ax=ax, color = '#FF5A5F')
    ax.set_title(var_name)
    
fig.tight_layout()

##### a. Host Listing Count

In [ ]:
plt.figure(figsize=(16,5))
plt.hist(listing['host_listings_count'],bins=30, color ='#FF5A5F')
plt.title('Distribution of Host Listings Count', fontsize=20)
plt.xlabel('Number of Listing a Host has', fontsize=15)
plt.ylabel('Number of Host', fontsize=15);

<div class="alert alert-block alert-success">

<b>Observation:</b>

Most host does not have any listing, and some has over 100 listings?!

</div>

In [ ]:
# Create new dataframe for neighbourhood count
neighbourhood_count = pd.DataFrame()
neighbourhood_count['total_hosts'] = listing['neighbourhood_cleansed'].value_counts()

# Filter out neighbourhood with less than 20 listings
neighbourhood_count = neighbourhood_count[neighbourhood_count['total_hosts'] > 20]

neighbourhood_count.sort_index(ascending=True, inplace=True)

In [ ]:
fig = plt.figure(1, figsize=(20,25))
ax = fig.add_subplot(111)

title="Distribution of listings based on neighbourhood in Tokyo"

transparency = 0.7
cmap = sns.color_palette("autumn")

neighbourhood_count['total_hosts'].plot(kind = 'barh',
                                        ax = ax,
                                        alpha = transparency,
                                        color = cmap,
                                        title = title)

ax.grid(False)
ax.set_frame_on(False)

ax.set_title(ax.get_title(), fontsize=26, alpha=transparency, ha='left')
plt.subplots_adjust(top=0.9)
ax.title.set_position((0,1.04))

ax.xaxis.set_label_position('top')
xlab = 'Total number of listings'
ax.set_xlabel(xlab, fontsize=20, alpha=transparency, ha='left')
ax.xaxis.set_label_coords(0, 1.02)
ax.xaxis.tick_top()

ylab = 'Neighbourhood in Tokyo'
ax.set_ylabel(ylab, fontsize=20, alpha=transparency);

##### a. Room_type

In [ ]:
room_type_count = pd.DataFrame()
room_type_count['total_rooms'] = listing['room_type'].value_counts()
room_type_count


In [ ]:
# room_type_count = pd.array(listing.room_type.value_counts())

In [ ]:
# room_type_count.head()

In [ ]:
room_type_count = pd.array(listing.room_type.value_counts())

# Customise plot color
colors = ['#FF5A5F', '#00A699', '#FC642D']
sns.set_palette(sns.color_palette(colors))

# Plot                
sns.catplot(x=listing['property_type'], y=room_type_count, hue=listing['room_type'], kind="bar", data=room_type_count);

In [ ]:
listing['time_since_first_review'] = listing.last_review - listing.first_review
listing.time_since_first_review.hist(figsize=(11,9), bins=30);

In [ ]:
fig = plt.figure(figsize=(11,9))
mask = np.triu(np.ones_like(listing.corr()))

# Customise plot color
cmap = sns.diverging_palette(220, 10, as_cmap=True)

ax = sns.heatmap(listing.corr(), 
                 annot=False, 
                 annot_kws={'size':10}, 
                 mask=mask,
                 cmap=cmap)

ax.set_title('Collinearity between Features', fontsize=20)
y_min, y_max = ax.get_ylim() 
x_min, x_max = ax.get_xlim()
ax.set_ylim(top=y_max+1) 
ax.set_xlim(right=x_max-1);

In [ ]:
fig = plt.figure(figsize=(11,9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(listing.corr()[['price']].sort_values(by='price', ascending=False), 
            cmap=cmap, 
            center=0, 
            annot=True)
plt.title('Correlation with price', fontsize=20)
y_min, y_max = ax.get_ylim() 
x_min, x_max = ax.get_xlim()
ax.set_ylim(top=y_max+1) 
ax.set_xlim(right=x_max-1);

In [ ]:
listing.dtypes

<div class="alert alert-block alert-warning">

<b>EDA PLOTS:</b> 

1. Distribution of Review scores 

2. Distribution of listings based on neighbourhood in Tokyo
    - total number of listing (y-axis)
    - neighbourhood in Tokyo (x-axis)
    
3. Popularity of room types in property types

4. see if there is any trend on bookings? (seasonality)

5. listings per host

6. property, room type on price
    

</div>

### <span style='background:yellow'> 5. Feature Engineering </span>

In [ ]:
listing = pd.get_dummies(listing, columns=['property_type',
                                           'room_type',
                                           'bed_type',
                                           'cancellation_policy', 
                                           'neighbourhood_group_cleansed',
                                           'host_response_time'], 
                         drop_first = False)

In [ ]:
all_amenities = listing['amenities'].str.get_dummies(sep=',')

In [ ]:
pd.concat([listing, all_amenities], axis=1)

<div class="alert alert-block alert-warning">

<b> Feature Engineering:</b>
- property_type
- room_type
- bed_type
- cancellation_policy
- neighbourhood_group_cleansed
- host_response_time

</div>